In [1]:
# environment set up
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Reshape, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import metrics
from tensorflow.keras import regularizers
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import accuracy_score
import os
import random
import pandas as pd
import ast
from scipy import stats as st
import time

In [45]:
spectrum_len = 250 # automate this
parent_dir = os.environ['PWD']
stddata_path = os.path.join(os.environ['DATA_DIR'], "StdData-" + str(spectrum_len))
os.chdir(os.path.join(parent_dir, "lab-notebook", "smunukutla"))

In [46]:
data = pd.read_csv("data.csv", sep=",")
record_nums = data.iloc[0, :].tolist()
spectrum_names = data.iloc[1, :].tolist()
y = data.iloc[2, :].astype(int).tolist()
y = np.reshape(y, (len(y), 1))
num_samples = len(y)

In [47]:
spectra = np.zeros((num_samples,spectrum_len))

In [48]:
for i in range(len(record_nums)):
    data = pd.read_csv(os.path.join(stddata_path,"{}.csv".format(record_nums[i])))
#     if i == 0:
#         wavelengths[i,:] = data.iloc[:, 0].to_numpy()
    spectra[i,:] = data.iloc[:, 1].to_numpy()

In [49]:
spectra.shape

(166, 250)

In [50]:
y_cat = to_categorical(y)

In [51]:
from sklearn.decomposition import FastICA

In [52]:
model = FastICA(n_components=3)

In [53]:
results = model.fit_transform(data)
results

/Users/Srikar/Desktop/Programs/Virtual Environments/spectra-ml-env/lib/python3.7/site-packages/sklearn/decomposition/fastica_.py:303: UserWarning: n_components is too large: it will be set to 2
  warnings.warn('n_components is too large: it will be set to %s' % n_components)


array([[ 0.09927671, -0.12984097],
       [ 0.10268178, -0.1295937 ],
       [ 0.09869894, -0.12794772],
       [ 0.09944543, -0.12719712],
       [ 0.09973701, -0.12636039],
       [ 0.09529167, -0.12462685],
       [ 0.09523576, -0.12372433],
       [ 0.09188562, -0.12219813],
       [ 0.08795886, -0.12056277],
       [ 0.08300581, -0.1187331 ],
       [ 0.0781878 , -0.116929  ],
       [ 0.07830101, -0.1160585 ],
       [ 0.07493836, -0.11452994],
       [ 0.06917948, -0.11254771],
       [ 0.06709529, -0.11126119],
       [ 0.06490981, -0.10995549],
       [ 0.06407948, -0.10890636],
       [ 0.06188943, -0.1075998 ],
       [ 0.06187929, -0.10670595],
       [ 0.05962176, -0.10538661],
       [ 0.05720334, -0.10403681],
       [ 0.05353843, -0.10245102],
       [ 0.0487156 , -0.10064601],
       [ 0.04666267, -0.0993654 ],
       [ 0.04282486, -0.09774688],
       [ 0.03495126, -0.09536428],
       [ 0.02795091, -0.09314701],
       [ 0.02062272, -0.09086767],
       [ 0.01696917,

In [54]:
results.shape

(250, 2)

In [19]:
def g(x):
    return np.tanh(x)
def g_der(x):
    return 1 - g(x) * g(x)

In [20]:
def center(X):
    X = np.array(X)
    
    mean = X.mean(axis=1, keepdims=True)
    
    return X- mean

In [21]:
def whitening(X):
    cov = np.cov(X)
    d, E = np.linalg.eigh(cov)
    D = np.diag(d)
    D_inv = np.sqrt(np.linalg.inv(D))
    X_whiten = np.dot(E, np.dot(D_inv, np.dot(E.T, X)))
    return X_whiten

In [22]:
def calculate_new_w(w, X):
    w_new = (X * g(np.dot(w.T, X))).mean(axis=1) - g_der(np.dot(w.T, X)).mean() * w
    w_new /= np.sqrt((w_new ** 2).sum())
    return w_new

In [28]:
def ica(X, iterations, tolerance=1e-5):
    X = center(X)
    
    X = whitening(X)
        
    components_nr = X.shape[0]
    
    W = np.zeros((components_nr, components_nr), dtype=X.dtype)

    for i in range(components_nr):
        
        w = np.random.rand(components_nr)
        
        for j in range(iterations):
            
            w_new = calculate_new_w(w, X)
            
            if i >= 1:
                w_new -= np.dot(np.dot(w_new, W[:i].T), W[:i])
            
            distance = np.abs(np.abs((w * w_new).sum()) - 1)
            
            w = w_new
            
            if distance < tolerance:
                break
                
        W[i, :] = w
        
    S = np.dot(W, X)
    
    return S

In [29]:
def plot_mixture_sources_predictions(X, original_sources, S):
    fig = plt.figure()
    plt.subplot(3, 1, 1)
    for x in X:
        plt.plot(x)
    plt.title("mixtures")
    plt.subplot(3, 1, 2)
    for s in original_sources:
        plt.plot(s)
    plt.title("real sources")
    plt.subplot(3,1,3)
    for s in S:
        plt.plot(s)
    plt.title("predicted sources")
    
    fig.tight_layout()
    plt.show()